## Part1: Toronto Neighborhood Project

In [44]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

#### Get the Postal Code data and create an empty data frame

In [45]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text

soup = BeautifulSoup(source, 'xml')
table=soup.find('table')
column_names=['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns=column_names)
df.head()

,Postalcode,Borough,Neighborhood


#### Process the data and add it to the data frame

In [46]:
for tr in table.find_all('tr'):
    row_data=[]
    for td in tr.find_all('td'):
        row_data.append(td.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Remove the rows with values "Not assigned"

In [47]:
df = df[df.Borough != "Not assigned"]
df.reset_index(drop=True, inplace=True) # drop the original index
df.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### Append the Neighborhood data for a all the Postal Codes and then drop the duplicate entries for repeated postalcodes

In [48]:
# Combine the Neighborhood values for the Postalcodes
temp_df=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhoods'},inplace=True)

# Merge the temp_df with original data frame on Postalcode 
toronto_df = pd.merge(df, temp_df, on='Postalcode')

# drop the column Neighborhood as we have the new values in Neighborhoods column
toronto_df.drop(['Neighborhood'],axis=1,inplace=True)

# drop the duplicate entries
toronto_df.drop_duplicates(inplace=True)

# Rename the Neighborhoods column that has the combined values ro Neighborhood
toronto_df.rename(columns={'Neighborhoods':'Neighborhood'},inplace=True)

# Reset the index
toronto_df.reset_index(drop=True, inplace=True) # drop the original index
toronto_df.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [49]:
toronto_df.shape

(103, 3)